In [ ]:
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tqdm

import src.data.Dataset as dt

In [ ]:
OUTPUT_FILE_PATH = os.path.abspath(
    "./resources/json/md_v5a/29618_images_0_003_threshold.json"
)
TRAIN_CSV = os.path.abspath("./data/processed/train/29618Train.csv")
# REPORT_CSV = os.path.abspath("./reports/model_coberture")

with open(OUTPUT_FILE_PATH, "r") as file:
    data = json.load(file)

dataset = dt.load_from_csv(TRAIN_CSV)

report_columns = [
    "file_name",
    "label",
    "threshold",
    "detector_label",
    "false_positive",
    "false_negative",
    "time_inference",
]
report = pd.DataFrame(columns=report_columns)

model = data["info"]["detector"].split(".")[0]
threshold = data["info"]["detector_threshold"]

for image in tqdm.tqdm(data["images"]):
    image_file = image["file"]
    indexes = dataset[dataset["file_name"] == image_file]

    if len(indexes) == 1:
        label = int(indexes["label"].iloc[0])
        detector_label = 1 if image["max_detection_conf"] > 0.0 else 0
        false_positive = int(label == 0 and detector_label == 1)
        false_negative = int(label == 1 and detector_label == 0)
        time_inference = image["time_inference"]

        new_row = {
            "file_name": image_file,
            "label": label,
            "threshold": threshold,
            "detector_label": detector_label,
            "false_positive": false_positive,
            "false_negative": false_negative,
            "time_inference": time_inference,
        }
        report = pd.concat([report, pd.DataFrame([new_row])], ignore_index=True)

# report_name = f"{len(data['images'])}_images_{model}.csv"
# REPORT_CSV = os.path.join(REPORT_CSV, report_name)

# dt.dataset_to_csv(report, REPORT_CSV)


In [ ]:
positivos_reales = report["label"].sum()
positivos_cubiertos = report[
    (report["label"] == 1) & (report["detector_label"] == 1)
].shape[0]

porcentaje_cubiertos = (
    (positivos_cubiertos / positivos_reales) * 100 if positivos_reales > 0 else 0
)

print(f"Hay {positivos_reales} positivos reales")
print(f"Se detectaron {positivos_cubiertos} positivos")
print(f"El porcentaje de positivos cubiertos es {porcentaje_cubiertos:.2f}%")
print(f"El porcentaje de positivos cubiertos es {porcentaje_cubiertos}")


In [ ]:
# Calcular métricas
verdaderos_positivos = len(
    report[(report["label"] == 1) & (report["detector_label"] == 1)]
)
falsos_positivos = len(report[(report["label"] == 0) & (report["detector_label"] == 1)])
verdaderos_negativos = len(
    report[(report["label"] == 0) & (report["detector_label"] == 0)]
)
falsos_negativos = len(report[(report["label"] == 1) & (report["detector_label"] == 0)])

precision = (
    verdaderos_positivos / (verdaderos_positivos + falsos_positivos)
    if (verdaderos_positivos + falsos_positivos) > 0
    else 0
)
exactitud = (
    (verdaderos_positivos + verdaderos_negativos) / len(report)
    if len(report) > 0
    else 0
)
tasa_verdaderos_positivos = (
    verdaderos_positivos / (verdaderos_positivos + falsos_negativos)
    if (verdaderos_positivos + falsos_negativos) > 0
    else 0
)
tasa_falsos_positivos = (
    falsos_positivos / (falsos_positivos + verdaderos_negativos)
    if (falsos_positivos + verdaderos_negativos) > 0
    else 0
)
tasa_verdaderos_negativos = (
    verdaderos_negativos / (verdaderos_negativos + falsos_positivos)
    if (verdaderos_negativos + falsos_positivos) > 0
    else 0
)
tasa_falsos_negativos = (
    falsos_negativos / (falsos_negativos + verdaderos_positivos)
    if (falsos_negativos + verdaderos_positivos) > 0
    else 0
)
f1_score = (
    2
    * (precision * tasa_verdaderos_positivos)
    / (precision + tasa_verdaderos_positivos)
    if (precision + tasa_verdaderos_positivos) > 0
    else 0
)

# Crear la matriz de confusión
cm = confusion_matrix(report["label"].astype(int), report["detector_label"].astype(int))

# Crear subplots
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Matriz de confusión
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Negativo", "Positivo"],
    yticklabels=["Negativo", "Positivo"],
    ax=axs[0],
)
axs[0].set_xlabel("MegaDetector")
axs[0].set_ylabel("Dataset")
axs[0].set_title("Matriz de Confusión")

# Gráfico de barras para las métricas
metrics = [
    "Precisión",
    "Exactitud",
    "Tasa de Verdaderos Positivos",
    "Tasa de Falsos Positivos",
    "Tasa de Verdaderos Negativos",
    "Tasa de Falsos Negativos",
    "F1 Score",
]
values = [
    precision,
    exactitud,
    tasa_verdaderos_positivos,
    tasa_falsos_positivos,
    tasa_verdaderos_negativos,
    tasa_falsos_negativos,
    f1_score,
]
axs[1].barh(metrics, values, color="skyblue")
axs[1].set_xlim(0, 1)
axs[1].set_title("Métricas de Evaluación")

# Ajustar el layout
plt.tight_layout()
plt.show()


In [ ]:
print("Valores de las métricas de evaluación:")
print(f"Precisión: {precision:.4f}")
print(f"Exactitud: {exactitud:.4f}")
print(f"Tasa de Verdaderos Positivos: {tasa_verdaderos_positivos:.4f}")
print(f"Tasa de Falsos Positivos: {tasa_falsos_positivos:.4f}")
print(f"Tasa de Verdaderos Negativos: {tasa_verdaderos_negativos:.4f}")
print(f"Tasa de Falsos Negativos: {tasa_falsos_negativos:.4f}")
print(f"F1 Score: {f1_score:.4f}")

In [ ]:
from PIL import Image
import os

# Ruta base
base_path = "/Users/carlos/WORKSPACE/MegaClassifier/dataset/datasetFiltered"

# Contadores para las imágenes que se pueden y no se pueden abrir
count_openable = 0
count_not_openable = 0

# Lista para almacenar las rutas de las imágenes que no se pueden abrir
not_openable_files = []

# Iterar sobre las rutas de las imágenes en el dataset
for image in data["images"]:
    file_path = image["file"]
    absolute_path = base_path + "/" + file_path
    try:
        # Intentar abrir la imagen
        img = Image.open(absolute_path)
        img.verify()  # Verificar que la imagen se puede abrir
        count_openable += 1
    except (IOError, SyntaxError) as e:
        # Si hay un error, incrementar el contador de imágenes no abiertas
        count_not_openable += 1
        not_openable_files.append(absolute_path)

# Imprimir los resultados
print(f"Number of openable images: {count_openable}")
print(f"Number of not openable images: {count_not_openable}")
print(f"Not openable files: {not_openable_files}")

Number of openable images: 29618
Number of not openable images: 0
Not openable files: []
